Let's build a simple neural network to classify images from the FashionMNIST dataset.

**1. Import Libraries**

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

*Checking for GPU Availability*

This code checks if a CUDA-enabled GPU is available and sets the `device` accordingly. If no GPU is available, it defaults to the CPU.

In [62]:
# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cuda


In [ ]:
# menit se muze:
# learning rate, batch size, pocet vrstev a pocet neuronu v nich, pocet epoch, dropout, pocet filtru/kanalu, padding, stride

# nejvyssi accuracy (90.2 %) se dosahlo pri nastaveni parametru nasledovne:
# BATCH_SIZE = 64, LR = 0.075, EPOCH = 10, DROPOUT = 0.15, OSTATNI SAME

**2. Data Preparation**

In [63]:
# Define a transform to convert images to tensors
transform = transforms.ToTensor()

# Download and load the training data
train_set = datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=64, shuffle=True)

# Download and load the test data
test_set = datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=64, shuffle=False)

**3. Neural Network Model**

In [64]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3)
        self.flat = nn.Flatten()
        self.fc1 = nn.Linear(in_features=24*24*64, out_features=128)
        self.drop = nn.Dropout(0.15)
        self.fc2 = nn.Linear(in_features=128, out_features=10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.flat(x)
        x = F.relu(self.fc1(x))
        x = self.drop(x)
        x = self.fc2(x)
        return x

model = SimpleCNN().to(device)
model

SimpleCNN(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (flat): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=36864, out_features=128, bias=True)
  (drop): Dropout(p=0.15, inplace=False)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)

**4. Loss & Optimizer**

In [65]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.075)

**5. Training loop**

In [66]:
for epoch in range(10):  # Train for _ epochs
    running_loss = 0.0
    for images, labels in train_loader:
        # Move images and labels to the device
        images = images.to(device)
        labels = labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f'Epoch [{epoch+1}/10], Loss: {running_loss / len(train_loader):.4f}')

Epoch [1/10], Loss: 0.6077
Epoch [2/10], Loss: 0.3780
Epoch [3/10], Loss: 0.3201
Epoch [4/10], Loss: 0.2833
Epoch [5/10], Loss: 0.2551
Epoch [6/10], Loss: 0.2287
Epoch [7/10], Loss: 0.2058
Epoch [8/10], Loss: 0.1872
Epoch [9/10], Loss: 0.1689
Epoch [10/10], Loss: 0.1493


**6. Evaluation on the test set**

In [67]:
correct = 0
total = 0
with torch.no_grad():  # Disable gradient calculation for evaluation
    for images, labels in test_loader:
        # Move images and labels to the device
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {100 * correct / total:.2f} %')

Accuracy: 90.22 %


In [ ]:
# PUVODNI PARAMETRY, BATCH_SIZE = 256, LR = 0.02, EPOCH = 5, KERNEL = 3, ...
"""
Epoch [1/5], Loss: 0.9816
Epoch [2/5], Loss: 0.6117
Epoch [3/5], Loss: 0.5575
Epoch [4/5], Loss: 0.5237
Epoch [5/5], Loss: 0.4974
Accuracy: 78.52 %
"""

# BATCH_SIZE = 128, OSTATNI SAME
"""
Epoch [1/5], Loss: 0.8287
Epoch [2/5], Loss: 0.5709
Epoch [3/5], Loss: 0.5148
Epoch [4/5], Loss: 0.4796
Epoch [5/5], Loss: 0.4544
Accuracy: 80.08 %
"""

# BATCH_SIZE = 64, OSTATNI SAME
"""
Epoch [1/5], Loss: 0.7269
Epoch [2/5], Loss: 0.5063
Epoch [3/5], Loss: 0.4426
Epoch [4/5], Loss: 0.4001
Epoch [5/5], Loss: 0.3696
Accuracy: 85.60 %
"""

# BATCH_SIZE = 64, LR = 0.002, OSTATNI SAME
"""
Epoch [1/5], Loss: 1.2327
Epoch [2/5], Loss: 0.6571
Epoch [3/5], Loss: 0.5848
Epoch [4/5], Loss: 0.5468
Epoch [5/5], Loss: 0.5184
Accuracy: 80.49 %
"""

# BATCH_SIZE = 64, LR = 0.002, EPOCH = 10, OSTATNI SAME
"""
Epoch [1/10], Loss: 1.3182
Epoch [2/10], Loss: 0.6681
Epoch [3/10], Loss: 0.5879
Epoch [4/10], Loss: 0.5436
Epoch [5/10], Loss: 0.5170
Epoch [6/10], Loss: 0.4948
Epoch [7/10], Loss: 0.4784
Epoch [8/10], Loss: 0.4654
Epoch [9/10], Loss: 0.4535
Epoch [10/10], Loss: 0.4419
Accuracy: 82.72 %
"""

# BATCH_SIZE = 64, LR = 0.005, EPOCH = 10, OSTATNI SAME
"""
Epoch [1/10], Loss: 0.9273
Epoch [2/10], Loss: 0.5728
Epoch [3/10], Loss: 0.5178
Epoch [4/10], Loss: 0.4903
Epoch [5/10], Loss: 0.4656
Epoch [6/10], Loss: 0.4506
Epoch [7/10], Loss: 0.4349
Epoch [8/10], Loss: 0.4259
Epoch [9/10], Loss: 0.4133
Epoch [10/10], Loss: 0.4073
Accuracy: 83.36 %
"""
# BATCH_SIZE = 64, LR = 0.005, EPOCH = 10, DROUPOUT = 0.5, OSTATNI SAME
"""
Epoch [1/10], Loss: 1.0222
Epoch [2/10], Loss: 0.6128
Epoch [3/10], Loss: 0.5595
Epoch [4/10], Loss: 0.5279
Epoch [5/10], Loss: 0.5062
Epoch [6/10], Loss: 0.4890
Epoch [7/10], Loss: 0.4747
Epoch [8/10], Loss: 0.4607
Epoch [9/10], Loss: 0.4539
Epoch [10/10], Loss: 0.4398
ccuracy: 82.78 %
"""

# BATCH_SIZE = 64, LR = 0.005, EPOCH = 10, DROUPOUT = 0.15, OSTATNI SAME
"""
Epoch [1/10], Loss: 0.8669
Epoch [2/10], Loss: 0.5521
Epoch [3/10], Loss: 0.4961
Epoch [4/10], Loss: 0.4619
Epoch [5/10], Loss: 0.4403
Epoch [6/10], Loss: 0.4237
Epoch [7/10], Loss: 0.4084
Epoch [8/10], Loss: 0.3974
Epoch [9/10], Loss: 0.3892
Epoch [10/10], Loss: 0.3755
Accuracy: 84.71 %
"""

# BATCH_SIZE = 64, LR = 0.005, EPOCH = 15, DROUPOUT = 0.15, OSTATNI SAME
"""
Epoch [1/15], Loss: 0.9376
Epoch [2/15], Loss: 0.5683
Epoch [3/15], Loss: 0.5113
Epoch [4/15], Loss: 0.4775
Epoch [5/15], Loss: 0.4556
Epoch [6/15], Loss: 0.4376
Epoch [7/15], Loss: 0.4213
Epoch [8/15], Loss: 0.4109
Epoch [9/15], Loss: 0.4017
Epoch [10/15], Loss: 0.3910
Epoch [11/15], Loss: 0.3840
Epoch [12/15], Loss: 0.3761
Epoch [13/15], Loss: 0.3693
Epoch [14/15], Loss: 0.3632
Epoch [15/15], Loss: 0.3572
Accuracy: 83.60 %
"""

# BATCH_SIZE = 64, LR = 0.075, EPOCH = 5, DROPOUT = 0.15, OSTATNI SAME
"""
Epoch [1/5], Loss: 0.6257
Epoch [2/5], Loss: 0.3858
Epoch [3/5], Loss: 0.3237
Epoch [4/5], Loss: 0.2844
Epoch [5/5], Loss: 0.2555
Accuracy: 89.31 %
"""

# BATCH_SIZE = 64, LR = 0.075, EPOCH = 10, DROPOUT = 0.15, OSTATNI SAME
"""
Epoch [1/10], Loss: 0.6077
Epoch [2/10], Loss: 0.3780
Epoch [3/10], Loss: 0.3201
Epoch [4/10], Loss: 0.2833
Epoch [5/10], Loss: 0.2551
Epoch [6/10], Loss: 0.2287
Epoch [7/10], Loss: 0.2058
Epoch [8/10], Loss: 0.1872
Epoch [9/10], Loss: 0.1689
Epoch [10/10], Loss: 0.1493
Accuracy: 90.22 %
"""

# Z PUVODNICH 78.5 % NA 90.2 %